In [1]:
import numpy as np
import math

create a function to calculate the distance between any row and the training data 


In [2]:
# function to calculate distance between two rows
def distance(row1, row2):
    # row[:-1] beacause the label shouldn't be included
    distance = (row2[:-1] - row1[:-1])**2
    sum = 0
    for i in distance:
        sum += i
    return math.sqrt(sum)

next will create a function to return an array containing the distances between a certain row and the whole training dataset

In [3]:
# find the distance array between the sample row and the complete dataset
# returns an array of distances
def distances_array(example, dataset):
    distances = np.empty((0,2), int)
    id_ = 0
    for row in dataset:
        distances = np.append(distances, np.array([[id_, distance(example, row)]]), axis=0) # must return id and distance
        id_ += 1
    return distances

to get the K-NN we need to sort the array resulting from the distances_array() and return the first K neighbors as an array

In [4]:
# k neighbours array 
# outputs the nearest K neighbors to a data example 
# returns an array with the nearest K neighbours and their id's
def k_neighbours(row, K, dataset):
    all_data = distances_array(row, dataset)
    sorted_all_data =  all_data[np.argsort(all_data[:,1])]
    KNN = sorted_all_data[:K]
    return KNN
    

the below predict function will use the row, and k_neighbours() to predict the outcome and return the accuracy:

In [5]:
def predict2(row, k, dataset_train):
    x = k_neighbours(row, k, dataset_train)
    ids = np.array([int(id[0]) for id in x])
    neighbors_labels = []
    for example in range(len(dataset_train)):
        for id_ in ids:
            if id_ == example:
                neighbors_labels.append(int(dataset_train[example][-1]))            
    true_label = row[-1]
    neighbors_labels_set = set(neighbors_labels)
    # we need to find p and accuracy
    # case 1: len(set(neighbour labels)) = 1
    if len(neighbors_labels_set) == 1:
        # neigb label consists only of true label
        if true_label in neighbors_labels_set:
            p = neighbors_labels[0]
            accuracy = 1
            return p, accuracy
        # same but tl is not in neigh label
        else:
            # len(neighbors_labels_set) == 1 and true_label not in neighbors_labels_set:
            p = neighbors_labels[0]
            accuracy = 0
            return p, accuracy
    # case 2: neighbor labels are mixed values
    else:
        nominees = find_ties(neighbors_labels)
        max_label = max(neighbors_labels, key = neighbors_labels.count)
        # pred label is in the neigh labels and not in the nominees then it has the majority
        if true_label == max_label and max_label not in nominees:
            p = max_label
            accuracy = 1
            return p, accuracy
        elif true_label != max_label and max_label not in nominees:
            p = max_label
            accuracy = 0
            return p, accuracy
        # tl is in nominees
        elif true_label in nominees:
            p = nominees
            accuracy = 1/len(p)
            return p, accuracy
        else:
            p = nominees
            accuracy = 0
            return p, accuracy
            

find_ties() is a helper function, that will examine the list of k_neighbours and retun a set of ties in the outcome

In [6]:
def find_ties(outcomes):
    # find number of labels in the final array
    unique_items, counts = np.unique(outcomes, return_counts=True)
    nominees = []
    k = 0
    for i in counts:
        m = 0
        for j in counts:
            if i == j and unique_items[k] != unique_items[m]:
                nominees.append(unique_items[k])
            m += 1
        k += 1
    return set(nominees)

## Start here

In [8]:
# load our training and testing sets
data_train = np.loadtxt('UCI_Dataset/pendigits_training.txt')
data_test = np.loadtxt('UCI_Dataset/pendigits_test.txt')
k=5

# normalize the data

data_train = (data_train - np.mean(data_train))/np.std(data_train)
data_test = (data_train - np.mean(data_train))/np.std(data_train)

result = []
id_=0
for row in data_test[:50]:
    p,a = predict2(row, k, data_train)
    result.append([id_, row[-1], p, a])
    id_ += 1
    # print something to show that its working
    if id_%20 == 0:
        print('reached id: ', id_)
# print results    
result

reached id:  20
reached id:  40


[[0, -1.134532074382495, -1, 0],
 [1, -1.304796484490872, -1, 0],
 [2, -1.3331738861756015, -1, 0],
 [3, -1.248041681121413, -1, 0],
 [4, -1.3331738861756015, -1, 0],
 [5, -1.1912868777519539, -1, 0],
 [6, -1.248041681121413, -1, 0],
 [7, -1.361551287860331, -1, 0],
 [8, -1.2196642794366834, -1, 0],
 [9, -1.361551287860331, -1, 0],
 [10, -1.1061546726977654, -1, 0],
 [11, -1.134532074382495, -1, 0],
 [12, -1.2196642794366834, -1, 0],
 [13, -1.1061546726977654, -1, 0],
 [14, -1.1629094760672245, -1, 0],
 [15, -1.2764190828061426, -1, 0],
 [16, -1.2764190828061426, -1, 0],
 [17, -1.1061546726977654, -1, 0],
 [18, -1.304796484490872, -1, 0],
 [19, -1.304796484490872, -1, 0],
 [20, -1.2196642794366834, -1, 0],
 [21, -1.3331738861756015, -1, 0],
 [22, -1.2196642794366834, -1, 0],
 [23, -1.134532074382495, -1, 0],
 [24, -1.1912868777519539, -1, 0],
 [25, -1.248041681121413, -1, 0],
 [26, -1.361551287860331, -1, 0],
 [27, -1.248041681121413, -1, 0],
 [28, -1.134532074382495, -1, 0],
 [29, -1.

In [25]:
# save output to file:
with open('5nn2.txt', 'w') as f:
    for item in result:
        f.write("%s\n" % item)

In [13]:
data_train[:-1]

array([[-0.02781341,  1.47618888, -0.59536144, ..., -0.22645522,
         1.41943408, -1.13453207],
       [-1.36155129,  1.16403746, -0.59536144, ...,  1.47618888,
        -1.19128688, -1.30479648],
       [-1.36155129,  0.25596061, -0.48185184, ..., -0.90751286,
        -1.36155129, -1.33317389],
       ...,
       [ 0.02894139,  1.47618888, -1.36155129, ..., -1.19128688,
        -1.36155129, -1.24804168],
       [ 1.47618888,  1.41943408,  0.34109281, ..., -1.36155129,
        -1.21966428, -1.21966428],
       [ 0.31271541,  0.48297982,  1.22079227, ...,  1.47618888,
        -1.36155129, -1.33317389]])

End 
